In [8]:
import asyncio
import websockets
import json
from datetime import datetime, timezone
import folium
import re
from IPython.display import display

# Lista globale per salvare le posizioni ricevute
navi_ricevute = []

# Crea la mappa (una sola volta)
mappa = folium.Map(location=[41.9, 12.5], zoom_start=2)  # zoom_start più ampio per vedere tutte le navi globali

def aggiungi_nave_a_mappa(riga_ais):
    match = re.search(r'ShipId:\s*(\d+)\s+Latitude:\s*(-?\d+\.\d+)\s+Longitude:\s*(-?\d+\.\d+)', riga_ais)
    if not match:
        print("Formato riga non valido")
        return

    mmsi = match.group(1)
    lat = float(match.group(2))
    lon = float(match.group(3))

    # Aggiungi solo se non è già stata inserita
    if mmsi not in [n["MMSI"] for n in navi_ricevute]:
        navi_ricevute.append({
            "MMSI": mmsi,
            "Latitude": lat,
            "Longitude": lon
        })

        folium.Marker(
            location=[lat, lon],
            popup=f"Nave MMSI: {mmsi}",
            icon=folium.Icon(color="blue", icon="ship", prefix="fa")
        ).add_to(mappa)

# Funzione asincrona per ricevere dati AIS
async def connect_ais_stream():
    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {
            "APIKey": "7d33927681ad1d125df5a6f75e0a90b7572aeb89",
            "BoundingBoxes": [[[ -90, -180 ], [ 90, 180 ]]],
            "FilterMessageTypes": ["PositionReport"]
        }

        await websocket.send(json.dumps(subscribe_message))

        count = 0
        async for message_json in websocket:
            message = json.loads(message_json)

            if message["MessageType"] == "PositionReport":
                ais = message["Message"]["PositionReport"]
                riga = f"[{datetime.now(timezone.utc)}] ShipId: {ais['UserID']} Latitude: {ais['Latitude']} Longitude: {ais['Longitude']}"
                print(riga)
                aggiungi_nave_a_mappa(riga)

                count += 1
                if count >= 20:  # puoi aumentare o rimuovere
                    break

# ⚙️ Esegui tutto in Jupyter così:
await connect_ais_stream()
display(mappa)  # Mostra UNA SOLA mappa alla fine


[2025-08-03 11:43:58.172945+00:00] ShipId: 235052117 Latitude: 51.49944833333333 Longitude: -0.02927
[2025-08-03 11:43:58.174945+00:00] ShipId: 226002270 Latitude: 49.49471166666667 Longitude: 0.18502000000000002
[2025-08-03 11:43:58.175975+00:00] ShipId: 730154797 Latitude: 11.020288333333333 Longitude: -74.78313499999999
[2025-08-03 11:43:58.194544+00:00] ShipId: 431006385 Latitude: 42.58073666666667 Longitude: 141.59568
[2025-08-03 11:43:58.195645+00:00] ShipId: 244670103 Latitude: 51.88399666666667 Longitude: 4.4216516666666665
[2025-08-03 11:43:58.195645+00:00] ShipId: 367047180 Latitude: 45.94701666666666 Longitude: -122.80673333333334
[2025-08-03 11:43:58.195645+00:00] ShipId: 244191618 Latitude: 51.91240166666667 Longitude: 4.24552
[2025-08-03 11:43:58.195645+00:00] ShipId: 244870700 Latitude: 53.893645 Longitude: 9.151305
[2025-08-03 11:43:58.208894+00:00] ShipId: 211232950 Latitude: 53.624135 Longitude: 7.161601666666667
[2025-08-03 11:43:58.208894+00:00] ShipId: 257178000 La